In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import requests
from bs4 import BeautifulSoup
import glob
import pandas as pd
import numpy as np
import string
import re
import nltk
nltk.download('punkt') # Download NLTK resources
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords') # Download the NLTK stop words
from nltk.tokenize import sent_tokenize
!pip install textstat
from textstat import textstat


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.0 MB/s eta 0:00:00


In [3]:
# Read the input Excel file
input_file = pd.read_excel(r"/content/drive/MyDrive/BlackCoffer Data Engineer Assignment/Provided files/Input.xlsx")

# Extract the title and text of article using the function
def extract_data(url):
    try:
        request = requests.get(url)
        soup = BeautifulSoup(request.text, "html.parser")
        title = soup.title.get_text()
        article_text = ""
        # check if the article content is in the class 'td-post-content'
        if soup.find("div", class_="td-post-content"):
           article_text = soup.find("div", class_="td-post-content").text.strip()
        # Check if the article content is in the class 'tdb-block-inner
        elif soup.find("div", class_="tdb-block-inner"):
            article_text = soup.find("div", class_="tdb-block-inner").text.strip()
        return title,article_text
    except Exception as e:
        print(f"An error occurred with URL {url}: {e}")
        return None, None
# Iterate over each row in the DataFrame
for index,row in input_file.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    # Extract data from the specified URL
    title, text = extract_data(url)
    # Save the extracted article text in a text file
    if text:
        with open(f'{url_id}.txt', 'w', encoding='utf-8') as file:
            file.write(f'Title: {title}\n\n')
            file.write(f'Text: {text}\n')
        print(f"Data extracted and saved for URL_ID: {url_id}")
    else:
        print(f"Data extraction failed for URL_ID: {url_id}")

print("Data extraction complete.")

Data extracted and saved for URL_ID: blackassign0001
Data extracted and saved for URL_ID: blackassign0002
Data extracted and saved for URL_ID: blackassign0003
Data extracted and saved for URL_ID: blackassign0004
Data extracted and saved for URL_ID: blackassign0005
Data extracted and saved for URL_ID: blackassign0006
Data extracted and saved for URL_ID: blackassign0007
Data extracted and saved for URL_ID: blackassign0008
Data extracted and saved for URL_ID: blackassign0009
Data extracted and saved for URL_ID: blackassign0010
Data extracted and saved for URL_ID: blackassign0011
Data extracted and saved for URL_ID: blackassign0012
Data extracted and saved for URL_ID: blackassign0013
Data extracted and saved for URL_ID: blackassign0014
Data extracted and saved for URL_ID: blackassign0015
Data extracted and saved for URL_ID: blackassign0016
Data extracted and saved for URL_ID: blackassign0017
Data extracted and saved for URL_ID: blackassign0018
Data extracted and saved for URL_ID: blackassi

In [4]:
import os

# Define the path to the specific folder in Google Drive where you want to save the files
folder_path = '/content/drive/My Drive/BlackCoffer Data Engineer Assignment/Extracted text files/'

# Make sure the folder exists, if not, create it
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Iterate over each row in the DataFrame
for index, row in input_file.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    # Extract data from the specified URL
    title, text = extract_data(url)
    # Save the extracted article text in a text file within the specified folder
    if text:
        file_path = os.path.join(folder_path, f'{url_id}.txt')
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(f'Title: {title}\n\n')
            file.write(f'Text: {text}\n')
        print(f"Data extracted and saved for URL_ID: {url_id} in folder: {folder_path}")
    else:
        print(f"Data extraction failed for URL_ID: {url_id}")

print("Data extraction complete.")


Data extracted and saved for URL_ID: blackassign0001 in folder: /content/drive/My Drive/BlackCoffer Data Engineer Assignment/Extracted text files/
Data extracted and saved for URL_ID: blackassign0002 in folder: /content/drive/My Drive/BlackCoffer Data Engineer Assignment/Extracted text files/
Data extracted and saved for URL_ID: blackassign0003 in folder: /content/drive/My Drive/BlackCoffer Data Engineer Assignment/Extracted text files/
Data extracted and saved for URL_ID: blackassign0004 in folder: /content/drive/My Drive/BlackCoffer Data Engineer Assignment/Extracted text files/
Data extracted and saved for URL_ID: blackassign0005 in folder: /content/drive/My Drive/BlackCoffer Data Engineer Assignment/Extracted text files/
Data extracted and saved for URL_ID: blackassign0006 in folder: /content/drive/My Drive/BlackCoffer Data Engineer Assignment/Extracted text files/
Data extracted and saved for URL_ID: blackassign0007 in folder: /content/drive/My Drive/BlackCoffer Data Engineer Assi

In [5]:
text_data=[]
# Iterate over each text file
for file in glob.glob('*.txt'):
    with open(file, 'r', encoding='latin-1') as files:
        title = ""
        text = ""
        # Flag to indicate if we are reading text content
        is_text = False
        # Counter for consecutive empty lines
        empty_line_count = 0
        for line in files:
            if line.strip().startswith("Title:"):
                title = line.strip().split('Title: ')[1]
            elif line.strip().startswith("Text:"):
                text = line.strip().split('Text: ')[1]
                is_text = True
            # Check if we are in the text section and the line is not empty
            elif is_text and line.strip():
                # Append the line to the text
                text += " " + line.strip()
                # Reset the empty line counter
                empty_line_count = 0
            # Check if we are in the text section and the line is empty
            elif is_text and not line.strip():
                # Increment the empty line counter
                empty_line_count += 1
                # If two consecutive lines are empty, stop reading text
                if empty_line_count == 2:
                    break
        # Extract the url_id from the file name
        url_id = file.split('.')[0]
        text_data.append({'URL_ID': url_id,'Title': title, 'Text': text})

# Create a DataFrame from the extracted data
extracted_df = pd.DataFrame(text_data)
print(extracted_df)

             URL_ID                                              Title  \
0   blackassign0084  How Voice search makes your business a success...   
1   blackassign0025  Rise of Electric Vehicles and its Impact on Li...   
2   blackassign0100  How will COVID-19 affect the world of work? - ...   
3   blackassign0040  How Machines, AI, Automations, and Robo-human ...   
4   blackassign0074  Gender diversity and Equality in the tech indu...   
..              ...                                                ...   
93  blackassign0048  How Data Analytics can help your business resp...   
94  blackassign0019  How does marketing influence businesses and co...   
95  blackassign0080  Impact of COVID-19 pandemic on sports events a...   
96  blackassign0095  What is the repercussion of the environment du...   
97  blackassign0023  Rising IT cities will impact the economy, envi...   

                                                 Text  
0   Finding ways to make using the Internet easier...  

In [6]:
extracted_df.head()

,URL_ID,Title,Text
0,blackassign0084,How Voice search makes your business a success...,Finding ways to make using the Internet easier...
1,blackassign0025,Rise of Electric Vehicles and its Impact on Li...,In 2020 India became the 5th largest automobil...
2,blackassign0100,How will COVID-19 affect the world of work? - ...,As business close to help prevent transmission...
3,blackassign0040,"How Machines, AI, Automations, and Robo-human ...",We all hear day in and day out that we amidst ...
4,blackassign0074,Gender diversity and Equality in the tech indu...,Gender diversity is an equal representation of...


In [7]:
text_data = []
# Iterate over each text file
for file in glob.glob('*.txt'):
    with open(file, 'r', encoding='latin-1') as files:
        title = ""
        text = ""
        # Flag to indicate if we are reading text content
        is_text = False
        # Counter for consecutive empty lines
        empty_line_count = 0
        for line in files:
            if line.strip().startswith("Title:"):
                title = line.strip().split('Title: ')[1]
            elif line.strip().startswith("Text:"):
                text = line.strip().split('Text: ')[1]
                is_text = True
            # Check if we are in the text section and the line is not empty
            elif is_text and line.strip():
                # Append the line to the text
                text += " " + line.strip()
                # Reset the empty line counter
                empty_line_count = 0
            # Check if we are in the text section and the line is empty
            elif is_text and not line.strip():
                # Increment the empty line counter
                empty_line_count += 1
                # If two consecutive lines are empty, stop reading text
                if empty_line_count == 2:
                    is_text = False
        # Extract the numeric part of the url_id from the file name
        url_id_numeric = int(file.split('blackassign')[1].split('.')[0])
        text_data.append({'URL_ID': url_id_numeric, 'Title': title, 'Text': text})

# Create a DataFrame from the extracted data
extracted_df = pd.DataFrame(text_data)

# Sort the DataFrame based on the numeric part of 'URL_ID'
extracted_df.sort_values('URL_ID', inplace=True)

# Convert the numeric part back to the original 'URL_ID' format with zero padding
extracted_df['URL_ID'] = 'blackassign' + extracted_df['URL_ID'].astype(str).str.zfill(4)

# Reset the index of the DataFrame
extracted_df.reset_index(drop=True, inplace=True)

print(extracted_df)


             URL_ID                                              Title  \
0   blackassign0001  Rising IT cities and its impact on the economy...   
1   blackassign0002  Rising IT Cities and Their Impact on the Econo...   
2   blackassign0003  Internet Demand's Evolution, Communication Imp...   
3   blackassign0004  Rise of Cybercrime and its Effect in upcoming ...   
4   blackassign0005  OTT platform and its impact on the entertainme...   
..              ...                                                ...   
93  blackassign0096  Due to the COVID-19 the repercussion of the en...   
94  blackassign0097  Impact of COVID-19 pandemic on office space an...   
95  blackassign0098  Contribution of handicrafts (Visual Arts & Lit...   
96  blackassign0099  How COVID-19 is impacting payment preferences?...   
97  blackassign0100  How will COVID-19 affect the world of work? - ...   

                                                 Text  
0   We have seen a huge development and dependence...  

In [8]:
extracted_df.head()

,URL_ID,Title,Text
0,blackassign0001,Rising IT cities and its impact on the economy...,We have seen a huge development and dependence...
1,blackassign0002,Rising IT Cities and Their Impact on the Econo...,"Throughout history, from the industrial revolu..."
2,blackassign0003,"Internet Demand's Evolution, Communication Imp...",Introduction In the span of just a few decades...
3,blackassign0004,Rise of Cybercrime and its Effect in upcoming ...,"The way we live, work, and communicate has unq..."
4,blackassign0005,OTT platform and its impact on the entertainme...,The year 2040 is poised to witness a continued...


In [9]:
# Load the stop words from files in the "StopWords" folder
stop_words = set()
stop_words_folder = '/content/drive/MyDrive/BlackCoffer Data Engineer Assignment/Provided files/StopWords folder/StopWords/'
for file_path in glob.glob(stop_words_folder + '*.txt'):
    with open(file_path, 'r', encoding='latin-1') as f:
        words = f.read().splitlines()
        # Convert words to lowercase before adding them to the set
        stop_words.update([word.lower() for word in words])


In [10]:
# Function for the clean_text
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Replace full stops with a special character
    text = text.replace('.', ' | ')
    # Remove extra spaces
    text = ' '.join(text.split())
    # Remove alphanumeric and numeric values
    text = re.sub(r'\w*\d\w*', '', text)
    # Remove special characters except the sentence boundary marker
    text = re.sub(r'[^a-zA-Z\s\|]', '', text)
    # Remove stopwords
    words = text.split()
    cleaned_words = [word for word in words if word.strip() not in stop_words]
    cleaned_text = ' '.join(cleaned_words)
    return cleaned_text
# Apply text cleaning to the 'Text' column in the DataFrame
extracted_df['Cleaned_Text'] = extracted_df['Text'].apply(clean_text)

In [11]:
# path to the stop words folder
stop_words_folder = r"/content/drive/MyDrive/BlackCoffer Data Engineer Assignment/Provided files/StopWords folder/StopWords/"

# Get the list of files in the stop words folder
stop_words_files = glob.glob(stop_words_folder + '*.txt')

# print the number of files and their names
print(f"Number of stop words files: {len(stop_words_files)}")
for file_path in stop_words_files:
    print(file_path)

Number of stop words files: 7
/content/drive/MyDrive/BlackCoffer Data Engineer Assignment/Provided files/StopWords folder/StopWords/StopWords_Geographic.txt
/content/drive/MyDrive/BlackCoffer Data Engineer Assignment/Provided files/StopWords folder/StopWords/StopWords_Names.txt
/content/drive/MyDrive/BlackCoffer Data Engineer Assignment/Provided files/StopWords folder/StopWords/StopWords_Auditor.txt
/content/drive/MyDrive/BlackCoffer Data Engineer Assignment/Provided files/StopWords folder/StopWords/StopWords_Currencies.txt
/content/drive/MyDrive/BlackCoffer Data Engineer Assignment/Provided files/StopWords folder/StopWords/StopWords_Generic.txt
/content/drive/MyDrive/BlackCoffer Data Engineer Assignment/Provided files/StopWords folder/StopWords/StopWords_GenericLong.txt
/content/drive/MyDrive/BlackCoffer Data Engineer Assignment/Provided files/StopWords folder/StopWords/StopWords_DatesandNumbers.txt


In [12]:
# Read the positive and negative words lists
positive_words = set()
with open(r"/content/drive/MyDrive/BlackCoffer Data Engineer Assignment/Provided files/MasterDictionary folder/MasterDictionary/positive-words.txt", 'r', encoding='latin-1') as f:
    positive_words = set(f.read().splitlines())

negative_words = set()
with open(r"/content/drive/MyDrive/BlackCoffer Data Engineer Assignment/Provided files/MasterDictionary folder/MasterDictionary/negative-words.txt", 'r', encoding='latin-1') as f:
    negative_words = set(f.read().splitlines())

# Assuming 'stop_words' is a set of stop words already defined
# Remove the stop words from positive and negative words lists
positive_words = positive_words - stop_words
negative_words = negative_words - stop_words

# Create a dictionary of positive and negative words
positive_dict = {word: 1 for word in positive_words}
negative_dict = {word: -1 for word in negative_words}

# Merge positive and negative dictionaries into a single sentiment dictionary
sentiment_dict = {**positive_dict, **negative_dict}


In [13]:
# Function to calculate the positive and negative scores
def calculate_scores(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())
    # Calculate the positive and negative scores
    positive_score = sum(1 for word in tokens if word in positive_words)
    negative_score = sum(1 for word in tokens if word in negative_words)
    return positive_score,negative_score
# Apply the function to the 'Text' column in the DataFrame
extracted_df['POSITIVE_SCORE'],extracted_df['NEGATIVE_SCORE'] = zip(*extracted_df['Cleaned_Text'].apply(calculate_scores))


In [14]:
# Function to calculate polarity score
def calculate_polarity_score(positive_score,negative_score):
    return (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
# Calculate Polarity Score for each row in the DataFrame
extracted_df['POLARITY_SCORE'] = extracted_df.apply(lambda row: calculate_polarity_score(row['POSITIVE_SCORE'], row['NEGATIVE_SCORE']), axis=1)


In [15]:
# Function to calculate subjectivity score
def calculate_subjectivity_score(positive_score, negative_score, cleaned_text):
    total_words = len(cleaned_text.split())
    return (positive_score + negative_score) / (total_words + 0.000001)
# Calculate Subjectivity Score for each row in the DataFrame
extracted_df['SUBJECTIVITY_SCORE'] = extracted_df.apply(lambda row: calculate_subjectivity_score(row['POSITIVE_SCORE'], row['NEGATIVE_SCORE'], row['Cleaned_Text']), axis=1)


In [16]:
# Function to calculate average sentence length
def calculate_average_sentence_length(text):
    sentences = sent_tokenize(text)
    Total_words = sum(len(word_tokenize(sentence)) for sentence in sentences)
    Total_sentences = len(sentences)
    return Total_words / Total_sentences


In [17]:
# Function to calcualte percentage of complex words
def calculate_percentage_complex_words(text):
    words = word_tokenize(text.lower())
    # Assuming words with more than 2 syllables are complex
    complex_words = [word for word in words if textstat.syllable_count(word) > 2]
    return len(complex_words) / len(words) if len(words) > 0 else 0


In [18]:
# Function to calculate Gunning Fox index
def calculate_gunning_fox_index(text):
    avg_sent_len = calculate_average_sentence_length(text)
    percentage_complex_words = calculate_percentage_complex_words(text)
    fog_index = 0.4 * (avg_sent_len + percentage_complex_words)
    return fog_index
# Apply the functions to the 'Text' column in the DataFrame
extracted_df['AVG_SENTENCE_LENGTH'] = extracted_df['Text'].apply(calculate_average_sentence_length)
extracted_df['PERCENTAGE_OF_COMPLEX_WORDS'] = extracted_df['Text'].apply(calculate_percentage_complex_words)
extracted_df['FOG_INDEX'] = extracted_df['Text'].apply(calculate_gunning_fox_index)


In [19]:
from nltk.tokenize import sent_tokenize, word_tokenize

def calculate_avg_words_per_sentence(text):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)
    # Calculate the total number of words
    total_words = len(word_tokenize(text))
    # Calculate the total number of sentences
    total_sentences = len(sentences)
    # Calculate the average number of words per sentence
    avg_words_per_sentence = total_words / total_sentences
    return avg_words_per_sentence
# Apply the function to the 'Text' column in the DataFrame
extracted_df['AVG_NUMBER_OF_WORD_PER_SENTENCE'] = extracted_df['Text'].apply(calculate_avg_words_per_sentence)


In [20]:
# Function to calculate complex word count
def complex_word_count(text):
    # Tokenize the text into words
    tokens = word_tokenize(text)
    # Count the complex words
    complex_words = [token for token in tokens if textstat.syllable_count(token)>2]
    return len(complex_words)
# Apply the function to the 'Text' column in the DataFrame
extracted_df['COMPLEX_WORD_COUNT'] = extracted_df['Text'].apply(complex_word_count)


In [21]:
# Function to remove stop words and count cleaned tokens
def word_count(text):
    # Tokenize the text into words
    tokens = word_tokenize(text)
    # Count the word count
    cleaned_tokens = [token for token in tokens if token not in stopwords.words('english')]
    # Count the cleaned tokens
    word_count = len(cleaned_tokens)
    return word_count
# Apply the function to the 'Text' column in the DataFrame
extracted_df['WORD_COUNT'] = extracted_df['Text'].apply(word_count)


In [22]:
# Function to syllable count per word
def count_syllables(word):
    vowels = "aeiouy"
    count = 0
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if word.endswith("le") and len(word) > 2 and word[-3] not in vowels:
        count += 1
    if word.endswith("es") or word.endswith("ed"):
        count -= 1
    if count == 0:
        count += 1
    return count
# Function to count syllables in each token
def count_syllables_in_tokens(tokens):
    # Tokenize the text into words
    Tokens = word_tokenize(text)
    # Count syllables in each token
    syllable_counts = [count_syllables(token) for token in tokens]
    return syllable_counts
# Apply the function to the 'Tokens' column in the DataFrame
extracted_df['SYLLABLE_PER_WORD'] = extracted_df['Text'].apply(count_syllables_in_tokens)


In [23]:
# Function to calculate personal pronouns count
def calculate_personal_pronouns_count(text):
    # Define the regex pattern
    pattern = r'\b(?:I|we|my|ours|us)\b'
    # Find all matches in the text
    matches = re.findall(pattern, text, flags=re.IGNORECASE)
    # Return the count of matches
    return len(matches)
# Apply the function to the 'Text' column in the DataFrame
extracted_df['PERSONAL_PRONOUNS'] = extracted_df['Text'].apply(calculate_personal_pronouns_count)

In [24]:
# Function to calculate the average word length
def calculate_average_word_length(tokens):
    # Tokenize the text into words
    tokens = word_tokenize(text)
    # Calculate the total number of characters in each word
    total_characters = sum(len(token) for token in tokens)
    # Calculate total number of words
    total_words= len(tokens)
    # Calculate the average word length
    average_word_length = total_characters / total_words if total_words else 0
    return average_word_length
# Apply the function to the 'Tokens' column in the DataFrame
extracted_df['AVG_WORD_LENGTH'] = extracted_df['Text'].apply(calculate_average_word_length)

In [25]:
# Assuming 'extracted_df' is your existing DataFrame with 'URL_ID', 'Title', and 'Text'
# Get the maximum URL_ID number from the DataFrame
max_url_id_num = extracted_df['URL_ID'].str.extract('(\d+)').astype(int).max()[0]
# Create a complete list of URL_IDs that should be present based on the maximum number found
complete_url_ids = ['blackassign' + str(i).zfill(4) for i in range(1, max_url_id_num + 1)]
# Convert the 'URL_ID' column to a list for comparison
existing_url_ids = extracted_df['URL_ID'].tolist()
# Identify missing URL_IDs
missing_url_ids = set(complete_url_ids) - set(existing_url_ids)
# Create rows with NaN values for the missing URL_IDs
missing_data = [{'URL_ID': url_id, 'Title': np.nan, 'Text': np.nan} for url_id in missing_url_ids]
# Convert the missing data to a DataFrame
missing_df = pd.DataFrame(missing_data)
# Append the missing data to the original DataFrame
complete_df = pd.concat([extracted_df, missing_df])
# Sort the DataFrame based on 'URL_ID'
complete_df.sort_values('URL_ID', inplace=True)
# Reset the index of the DataFrame
complete_df.reset_index(drop=True, inplace=True)

print(complete_df)


             URL_ID                                              Title  \
0   blackassign0001  Rising IT cities and its impact on the economy...   
1   blackassign0002  Rising IT Cities and Their Impact on the Econo...   
2   blackassign0003  Internet Demand's Evolution, Communication Imp...   
3   blackassign0004  Rise of Cybercrime and its Effect in upcoming ...   
4   blackassign0005  OTT platform and its impact on the entertainme...   
..              ...                                                ...   
95  blackassign0096  Due to the COVID-19 the repercussion of the en...   
96  blackassign0097  Impact of COVID-19 pandemic on office space an...   
97  blackassign0098  Contribution of handicrafts (Visual Arts & Lit...   
98  blackassign0099  How COVID-19 is impacting payment preferences?...   
99  blackassign0100  How will COVID-19 affect the world of work? - ...   

                                                 Text  \
0   We have seen a huge development and dependence... 

In [26]:
complete_df=complete_df.drop(columns=['Title','Text','Cleaned_Text'])

In [27]:
complete_df.loc[35]

URL_ID                             blackassign0036
POSITIVE_SCORE                                 NaN
NEGATIVE_SCORE                                 NaN
POLARITY_SCORE                                 NaN
SUBJECTIVITY_SCORE                             NaN
AVG_SENTENCE_LENGTH                            NaN
PERCENTAGE_OF_COMPLEX_WORDS                    NaN
FOG_INDEX                                      NaN
AVG_NUMBER_OF_WORD_PER_SENTENCE                NaN
COMPLEX_WORD_COUNT                             NaN
WORD_COUNT                                     NaN
SYLLABLE_PER_WORD                              NaN
PERSONAL_PRONOUNS                              NaN
AVG_WORD_LENGTH                                NaN
Name: 35, dtype: object

In [28]:
complete_df.loc[48]

URL_ID                             blackassign0049
POSITIVE_SCORE                                 NaN
NEGATIVE_SCORE                                 NaN
POLARITY_SCORE                                 NaN
SUBJECTIVITY_SCORE                             NaN
AVG_SENTENCE_LENGTH                            NaN
PERCENTAGE_OF_COMPLEX_WORDS                    NaN
FOG_INDEX                                      NaN
AVG_NUMBER_OF_WORD_PER_SENTENCE                NaN
COMPLEX_WORD_COUNT                             NaN
WORD_COUNT                                     NaN
SYLLABLE_PER_WORD                              NaN
PERSONAL_PRONOUNS                              NaN
AVG_WORD_LENGTH                                NaN
Name: 48, dtype: object

In [29]:
print(input_file)
input_data = pd.DataFrame(input_file)

             URL_ID                                                URL
0   blackassign0001  https://insights.blackcoffer.com/rising-it-cit...
1   blackassign0002  https://insights.blackcoffer.com/rising-it-cit...
2   blackassign0003  https://insights.blackcoffer.com/internet-dema...
3   blackassign0004  https://insights.blackcoffer.com/rise-of-cyber...
4   blackassign0005  https://insights.blackcoffer.com/ott-platform-...
..              ...                                                ...
95  blackassign0096  https://insights.blackcoffer.com/what-is-the-r...
96  blackassign0097  https://insights.blackcoffer.com/impact-of-cov...
97  blackassign0098  https://insights.blackcoffer.com/contribution-...
98  blackassign0099  https://insights.blackcoffer.com/how-covid-19-...
99  blackassign0100  https://insights.blackcoffer.com/how-will-covi...

[100 rows x 2 columns]


In [30]:
output_data_structure = pd.merge(input_data,complete_df,on='URL_ID')

In [31]:
output_data_structure.head()

,URL_ID,URL,POSITIVE_SCORE,NEGATIVE_SCORE,POLARITY_SCORE,SUBJECTIVITY_SCORE,AVG_SENTENCE_LENGTH,PERCENTAGE_OF_COMPLEX_WORDS,FOG_INDEX,AVG_NUMBER_OF_WORD_PER_SENTENCE,COMPLEX_WORD_COUNT,WORD_COUNT,SYLLABLE_PER_WORD,PERSONAL_PRONOUNS,AVG_WORD_LENGTH
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,26.0,6.0,0.625000,0.057866,17.986842,0.096822,7.233466,17.986842,131.0,889.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",12.0,4.673294
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,51.0,31.0,0.243902,0.108609,21.037975,0.192539,8.492206,21.037975,320.0,1117.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",6.0,4.673294
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,36.0,23.0,0.220339,0.090076,21.263158,0.254125,8.606913,21.263158,308.0,860.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",13.0,4.673294
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,36.0,74.0,-0.345455,0.174326,23.250000,0.243355,9.397342,23.250000,293.0,858.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",5.0,4.673294
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,19.0,8.0,0.407407,0.072581,19.461538,0.171278,7.853127,19.461538,130.0,516.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",6.0,4.673294


In [32]:
!pip install openpyxl

In [33]:
# Save the DataFrame to an Excel file
output_data_structure.to_excel('/content/drive/MyDrive/BlackCoffer Data Engineer Assignment/output_data.xlsx', index=False)